In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("victorcallejasf/multimodal-hate-speech")

print("Path to dataset files:", path)

In [13]:
cd /root/.cache/kagglehub/datasets/victorcallejasf/multimodal-hate-speech/versions/1

/root/.cache/kagglehub/datasets/victorcallejasf/multimodal-hate-speech/versions/1


/usr/local/lib/python3.11/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [14]:
ll

total 61220
-rw-r--r-- 1 root 51426208 Feb 23 14:31 MMHS150K_GT.json
-rw-r--r-- 1 root     1079 Feb 23 14:31 MMHS150K_readme.txt
-rw-r--r-- 1 root      952 Feb 23 14:31 hatespeech_keywords.txt
drwxr-xr-x 2 root  6144000 Feb 23 14:32 img_resized/
drwxr-xr-x 2 root  2428928 Feb 23 14:32 img_txt/
drwxr-xr-x 2 root       82 Feb 23 14:32 splits/


In [ ]:
cd /workspace/CookBook/chapter09-Multi_Modal

In [4]:
import pandas as pd 

df = pd.read_csv("/workspace/CookBook/chapter09-Multi_Modal/final_df.csv")

약 6G정도의 데이터를 다운 받습니다. 

약 1분정도 소요됩니다. 

In [5]:
df.head(2)

,index,file_id,img_url,labels,tweet_url,tweet_text,labels_str,file_path,is_hate,original,translated
0,11872,1106904873637953537,http://pbs.twimg.com/tweet_video_thumb/D1yEG7V...,"[0, 0, 0]",https://twitter.com/user/status/11069048736379...,Also woke up to this nigga asking me out on a ...,"['NotHate', 'NotHate', 'NotHate']",./img_resized/1106904873637953537.jpg,False,Also woke up to this nigga asking me out on a ...,또 이 사람한테 데이트 신청 받는 걸로 일어났어... 하지만 아니야. 난 더 이상 ...
1,381,1108495255819730945,http://pbs.twimg.com/media/D2Iqj05X0AAzksK.jpg,"[0, 0, 0]",https://twitter.com/user/status/11084952558197...,#RNS REAL NIGGA SHIT,"['NotHate', 'NotHate', 'NotHate']",./img_resized/1108495255819730945.jpg,False,#RNS REAL NIGGA SHIT,#RNS 진짜 새끼들 일


In [6]:
df["is_hate"] = df["is_hate"].astype(int)
df["file_path"] = df["file_path"].map(lambda x : x.replace("./", "/root/.cache/kagglehub/datasets/victorcallejasf/multimodal-hate-speech/versions/1/"))
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [7]:
from datasets import Dataset
hf_dataset = Dataset.from_pandas(df)

In [8]:
prompt = """입력으로 이미지 또는 ###텍스트가 주어지면, 해당 콘텐츠가 유해한지 여부를 "1" 또는 "0"으로 출력하세요.
- 출력은 반드시 숫자 하나만(문자열 없이) 출력합니다.
  - "1" : 유해(혐오·차별·폭력·선정성·기타 해악 가능성이 있다고 판단되는 경우)
  - "0" : 비유해(문제 없다고 판단되는 경우)
유해성(harmfulness)을 판단할 때는 아래 항목을 포함해 검토하세요:
1. **혐오 표현 (Hate Speech)**  
   - 인종, 성별, 종교, 성적 지향, 장애 등을 근거로 특정 집단/개인을 비하·모욕·차별하는 언어/이미지
   - 명백한 증오, 차별, 폭력 조장 등
2. **직접적인 폭력·잔혹 행위**  
   - 극단적인 폭력 묘사(살해·자해·테러 등)
   - 노골적으로 잔혹하거나 충격적인 내용
3. **노골적 성적 콘텐츠**  
   - 명백히 선정적·음란한 묘사, 아동 성착취 등 불법적이거나 심각한 윤리적 문제가 있는 경우
4. **사생활 침해, 불법 행위 조장**  
   - 명백한 사생활 침해, 범죄 또는 불법 행위를 조장하는 내용
5. **기타 위험 요소**  
   - 위 항목 외에도, 사회적으로 유해하거나 심각한 해악을 유발할 수 있는 이미지·텍스트
만약 콘텐츠가 위 기준 중 하나라도 충족하여 "유해"하다고 판단되면 "1"을,  
해당되지 않으면 "0"을 출력하십시오.
출력은 오직 숫자 하나(1 또는 0)만 반환하고, 어떠한 추가 문구나 설명도 첨부하지 마십시오.

###텍스트 :{text}"""

system_message = "당신은 이미지와 텍스트의 '유해성(harmfulness)'을 판별하는 분류 모델입니다."

In [9]:
from datasets import load_dataset

# 데이터셋을 OpenAI 메시지 형식으로 변환하는 함수      
def format_data(sample):
   return {"messages": [
               {
                   "role": "system", # 시스템 역할
                   "content": [{"type": "text", "text": system_message}], # 시스템 메시지
               },
               {
                   "role": "user",  # 사용자 역할
                   "content": [
                       {
                           "type": "text",
                           # 제품명과 카테고리를 포함한 프롬프트 생성
                           "text": prompt.format(text=sample["translated"]),
                       },{
                           "type": "image", # 이미지 타입
                           "image": sample["file_path"], # 제품 이미지
                       }
                   ],
               },
               {
                   "role": "assistant", # AI 어시스턴트 역할
                   "content": [{"type": "text", "text": sample["is_hate"]}], # 제품 설명
               },
           ],
       }


dataset = [format_data(sample) for sample in hf_dataset]

In [10]:
dataset[0]

{'messages': [{'role': 'system',
   'content': [{'type': 'text',
     'text': "당신은 이미지와 텍스트의 '유해성(harmfulness)'을 판별하는 분류 모델입니다."}]},
  {'role': 'user',
   'content': [{'type': 'text',
     'text': '입력으로 이미지 또는 ###텍스트가 주어지면, 해당 콘텐츠가 유해한지 여부를 "1" 또는 "0"으로 출력하세요.\n- 출력은 반드시 숫자 하나만(문자열 없이) 출력합니다.\n  - "1" : 유해(혐오·차별·폭력·선정성·기타 해악 가능성이 있다고 판단되는 경우)\n  - "0" : 비유해(문제 없다고 판단되는 경우)\n유해성(harmfulness)을 판단할 때는 아래 항목을 포함해 검토하세요:\n1. **혐오 표현 (Hate Speech)**  \n   - 인종, 성별, 종교, 성적 지향, 장애 등을 근거로 특정 집단/개인을 비하·모욕·차별하는 언어/이미지\n   - 명백한 증오, 차별, 폭력 조장 등\n2. **직접적인 폭력·잔혹 행위**  \n   - 극단적인 폭력 묘사(살해·자해·테러 등)\n   - 노골적으로 잔혹하거나 충격적인 내용\n3. **노골적 성적 콘텐츠**  \n   - 명백히 선정적·음란한 묘사, 아동 성착취 등 불법적이거나 심각한 윤리적 문제가 있는 경우\n4. **사생활 침해, 불법 행위 조장**  \n   - 명백한 사생활 침해, 범죄 또는 불법 행위를 조장하는 내용\n5. **기타 위험 요소**  \n   - 위 항목 외에도, 사회적으로 유해하거나 심각한 해악을 유발할 수 있는 이미지·텍스트\n만약 콘텐츠가 위 기준 중 하나라도 충족하여 "유해"하다고 판단되면 "1"을,  \n해당되지 않으면 "0"을 출력하십시오.\n출력은 오직 숫자 하나(1 또는 0)만 반환하고, 어떠한 추가 문구나 설명도 첨부하지 마십시오.\n\n###텍스트 :시발 그녀'},
    {'type': 'image',


In [11]:
from sklearn.model_selection import train_test_split

# test_size=0.1로 설정하여 전체 데이터의 10%를 테스트 세트로 분리
train_dataset, test_dataset = train_test_split(dataset, 
                                             test_size=0.1, 
                                             random_state=42)

In [12]:
train_dataset[0]

{'messages': [{'role': 'system',
   'content': [{'type': 'text',
     'text': "당신은 이미지와 텍스트의 '유해성(harmfulness)'을 판별하는 분류 모델입니다."}]},
  {'role': 'user',
   'content': [{'type': 'text',
     'text': '입력으로 이미지 또는 ###텍스트가 주어지면, 해당 콘텐츠가 유해한지 여부를 "1" 또는 "0"으로 출력하세요.\n- 출력은 반드시 숫자 하나만(문자열 없이) 출력합니다.\n  - "1" : 유해(혐오·차별·폭력·선정성·기타 해악 가능성이 있다고 판단되는 경우)\n  - "0" : 비유해(문제 없다고 판단되는 경우)\n유해성(harmfulness)을 판단할 때는 아래 항목을 포함해 검토하세요:\n1. **혐오 표현 (Hate Speech)**  \n   - 인종, 성별, 종교, 성적 지향, 장애 등을 근거로 특정 집단/개인을 비하·모욕·차별하는 언어/이미지\n   - 명백한 증오, 차별, 폭력 조장 등\n2. **직접적인 폭력·잔혹 행위**  \n   - 극단적인 폭력 묘사(살해·자해·테러 등)\n   - 노골적으로 잔혹하거나 충격적인 내용\n3. **노골적 성적 콘텐츠**  \n   - 명백히 선정적·음란한 묘사, 아동 성착취 등 불법적이거나 심각한 윤리적 문제가 있는 경우\n4. **사생활 침해, 불법 행위 조장**  \n   - 명백한 사생활 침해, 범죄 또는 불법 행위를 조장하는 내용\n5. **기타 위험 요소**  \n   - 위 항목 외에도, 사회적으로 유해하거나 심각한 해악을 유발할 수 있는 이미지·텍스트\n만약 콘텐츠가 위 기준 중 하나라도 충족하여 "유해"하다고 판단되면 "1"을,  \n해당되지 않으면 "0"을 출력하십시오.\n출력은 오직 숫자 하나(1 또는 0)만 반환하고, 어떠한 추가 문구나 설명도 첨부하지 마십시오.\n\n###텍스트 :이보다 더 멍청한 걸 본 적 있으면 나한테 연락해:  #

In [11]:
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor, BitsAndBytesConfig

# 허깅페이스 모델 ID
model_id = "Qwen/Qwen2-VL-7B-Instruct" 

# 모델과 프로세서 로드
model = AutoModelForVision2Seq.from_pretrained(
   model_id,
   device_map="auto",                            # GPU 메모리에 자동 할당
   # attn_implementation="flash_attention_2",     # 학습시에는 flash attention 2 미지원
   torch_dtype=torch.bfloat16,                   # bfloat16 정밀도 사용
)
processor = AutoProcessor.from_pretrained(model_id)  # 텍스트/이미지 전처리기 로드

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [16]:
text = processor.apply_chat_template(
    train_dataset[2]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|im_start|>system
당신은 이미지와 텍스트의 '유해성(harmfulness)'을 판별하는 분류 모델입니다.<|im_end|>
<|im_start|>user
입력으로 이미지 또는 ###텍스트가 주어지면, 해당 콘텐츠가 유해한지 여부를 "1" 또는 "0"으로 출력하세요.
- 출력은 반드시 숫자 하나만(문자열 없이) 출력합니다.
  - "1" : 유해(혐오·차별·폭력·선정성·기타 해악 가능성이 있다고 판단되는 경우)
  - "0" : 비유해(문제 없다고 판단되는 경우)
유해성(harmfulness)을 판단할 때는 아래 항목을 포함해 검토하세요:
1. **혐오 표현 (Hate Speech)**  
   - 인종, 성별, 종교, 성적 지향, 장애 등을 근거로 특정 집단/개인을 비하·모욕·차별하는 언어/이미지
   - 명백한 증오, 차별, 폭력 조장 등
2. **직접적인 폭력·잔혹 행위**  
   - 극단적인 폭력 묘사(살해·자해·테러 등)
   - 노골적으로 잔혹하거나 충격적인 내용
3. **노골적 성적 콘텐츠**  
   - 명백히 선정적·음란한 묘사, 아동 성착취 등 불법적이거나 심각한 윤리적 문제가 있는 경우
4. **사생활 침해, 불법 행위 조장**  
   - 명백한 사생활 침해, 범죄 또는 불법 행위를 조장하는 내용
5. **기타 위험 요소**  
   - 위 항목 외에도, 사회적으로 유해하거나 심각한 해악을 유발할 수 있는 이미지·텍스트
만약 콘텐츠가 위 기준 중 하나라도 충족하여 "유해"하다고 판단되면 "1"을,  
해당되지 않으면 "0"을 출력하십시오.
출력은 오직 숫자 하나(1 또는 0)만 반환하고, 어떠한 추가 문구나 설명도 첨부하지 마십시오.

###텍스트 :생일 축하해, 내 새끼 🎉🎉🎉<|vision_start|><|image_pad|><|vision_end|><|im_end|>
<|im_start|>assistant
1<|im_end|>



In [14]:
from qwen_vl_utils import process_vision_info

# 모델 답변을 생성하는 함수
def generate_description(messages, model, processor):
   # 추론을 위한 준비
   text = processor.apply_chat_template(
       messages, tokenize=False, add_generation_prompt=True
   )
   image_inputs, video_inputs = process_vision_info(messages)
   inputs = processor(
       text=[text],
       images=image_inputs,
       videos=video_inputs,
       padding=True,
       return_tensors="pt",
   )
   inputs = inputs.to(model.device)
   # 추론: 출력 생성
   generated_ids = model.generate(
      **inputs, 
      max_new_tokens=128,
      top_p=0.95, 
      do_sample=True, 
      temperature=0.1
      )
   generated_ids_trimmed = [
      out_ids[len(in_ids) :] 
      for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
   output_text = processor.batch_decode(
       generated_ids_trimmed, 
       skip_special_tokens=True, 
       clean_up_tokenization_spaces=False
   )
   return output_text[0]

In [18]:
messages =  test_dataset[2]["messages"]
answer = test_dataset[2]["messages"][2]["content"][0]["text"]
base_description = generate_description(messages, model, processor)
print("예측 결과 :", base_description)
print("정답 결과 :", answer)

예측 결과 : 1
정답 결과 : 1


In [19]:
from tqdm.auto import tqdm 
no_train_result = [] 

for idx in tqdm(range(len(test_dataset))):
    messages =  test_dataset[idx]["messages"]
    answer = test_dataset[idx]["messages"][2]["content"][0]["text"]
    base_description = generate_description(messages, model, processor)
    no_train_result.append((answer, base_description))

  0%|          | 0/240 [00:00<?, ?it/s]

In [20]:
prediction_result = [int(predict) == int(answer) for predict, answer in no_train_result]
before_train_accuracy = sum(prediction_result) /  len(prediction_result) * 100
print(f"학습 전 정확도 : {before_train_accuracy:.2f}%") 

학습 전 정확도 : 67.08%


In [22]:
from peft import LoraConfig

peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.05,
        r=8,
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM", 
)

In [23]:
from trl import SFTConfig
from transformers import Qwen2VLProcessor
from qwen_vl_utils import process_vision_info

# SFTConfig를 통해 학습 설정을 정의
args = SFTConfig(
    output_dir="qwen2-7b-instruct-harmful-detector-2000",   # 학습된 모델과 체크포인트를 저장할 디렉터리 경로 및 리포지토리 ID
    num_train_epochs=1,                                # 전체 학습 에포크 수 (데이터셋을 몇 번 반복할지 설정)
    per_device_train_batch_size=8,                     # 각 장비(GPU)당 사용될 배치 사이즈 (메모리와 연관됨)
    gradient_accumulation_steps=8,                     # 경사 누적 스텝 수 (이 횟수만큼 기울기를 누적한 후 업데이트)
    gradient_checkpointing=True,                       # 메모리 절약을 위한 gradient checkpointing 활성화 (메모리 최적화)
    optim="adamw_torch_fused",                         # AdamW 옵티마이저 (fused 버전 사용으로 학습 속도 향상)
    logging_steps=5,                                   # 몇 스텝마다 로그를 출력할지 설정 (여기선 5 스텝마다 로그)
    save_strategy="epoch",                             # 매 에포크마다 체크포인트 저장 설정
    learning_rate=2e-4,                                # 학습률 (QLoRA 논문에서 추천된 값 사용)
    bf16=True,                                         # bfloat16 정밀도 사용 (메모리 절약 및 속도 향상)
    tf32=True,                                         # tf32 정밀도 사용 (NVIDIA GPU에서 학습 속도 향상)
    max_grad_norm=0.3,                                 # 기울기 클리핑을 위한 최대 기울기 값 (QLoRA 논문에서 추천된 값)
    warmup_ratio=0.03,                                 # 학습 초기에 학습률을 점진적으로 올리는 warmup 비율 (QLoRA 논문에서 추천된 값)
    lr_scheduler_type="constant",                      # 일정한 학습률 스케줄러 사용 (학습률이 변하지 않음)
    # push_to_hub=True,                                # 학습된 모델을 Hugging Face Hub에 푸시할지 여부
    report_to="tensorboard",                           # TensorBoard를 통해 학습 상태를 모니터링
    gradient_checkpointing_kwargs={"use_reentrant": False}, # reentrant gradient checkpointing 설정 (비재진입 방식 사용)
    dataset_text_field="",                             # 데이터셋에서 텍스트 필드를 위한 더미 필드 (collator에서 필요)
    dataset_kwargs={"skip_prepare_dataset": True}      # collator에서 데이터셋 전처리를 건너뛰기 위한 설정
)

# 불필요한 열 삭제하지 않도록 설정 (학습 중 사용되지 않는 열이라도 유지)
args.remove_unused_columns = False

In [24]:
# 텍스트와 이미지 쌍을 인코딩하기 위한 데이터 collator 함수 정의
def collate_fn(examples):
    # 각 예제에서 텍스트와 이미지를 추출하고, 텍스트는 채팅 템플릿을 적용
    texts = [processor.apply_chat_template(example["messages"], tokenize=False) for example in examples]
    image_inputs = [process_vision_info(example["messages"])[0] for example in examples]

    # 텍스트를 토크나이징하고 이미지를 처리하여 일괄 처리(batch) 형태로 변환
    batch = processor(text=texts, images=image_inputs, return_tensors="pt", padding=True)

    # labels로 사용할 input_ids 복사본 생성 후, 패딩 토큰을 -100으로 설정하여 손실 계산 시 무시하도록 함
    labels = batch["input_ids"].clone()
    labels[labels == processor.tokenizer.pad_token_id] = -100  # 패딩 토큰 손실 계산 제외

    # 특정 이미지 토큰 인덱스는 손실 계산에서 무시 (모델에 따라 다름)
    if isinstance(processor, Qwen2VLProcessor):  
        # Qwen2VL 모델의 이미지 토큰 인덱스
        image_tokens = [151652, 151653, 151655]
    else:
        # 다른 모델에서 이미지 토큰 ID를 얻어 손실 계산에서 제외
        image_tokens = [processor.tokenizer.convert_tokens_to_ids(processor.image_token)]
    
    # 손실 계산 시 이미지 토큰 인덱스를 무시하도록 설정
    for image_token_id in image_tokens:
        labels[labels == image_token_id] = -100
    
    # 배치에 labels 추가 (손실 계산 시 사용)
    batch["labels"] = labels

    return batch

In [25]:
# 단일 예시 확인
example = dataset[0]  # 데이터셋의 첫 번째 아이템
print("단일 예시 데이터:")
print(example)

단일 예시 데이터:
{'messages': [{'role': 'system', 'content': [{'type': 'text', 'text': "당신은 이미지와 텍스트의 '유해성(harmfulness)'을 판별하는 분류 모델입니다."}]}, {'role': 'user', 'content': [{'type': 'text', 'text': '입력으로 이미지 또는 ###텍스트가 주어지면, 해당 콘텐츠가 유해한지 여부를 "1" 또는 "0"으로 출력하세요.\n- 출력은 반드시 숫자 하나만(문자열 없이) 출력합니다.\n  - "1" : 유해(혐오·차별·폭력·선정성·기타 해악 가능성이 있다고 판단되는 경우)\n  - "0" : 비유해(문제 없다고 판단되는 경우)\n유해성(harmfulness)을 판단할 때는 아래 항목을 포함해 검토하세요:\n1. **혐오 표현 (Hate Speech)**  \n   - 인종, 성별, 종교, 성적 지향, 장애 등을 근거로 특정 집단/개인을 비하·모욕·차별하는 언어/이미지\n   - 명백한 증오, 차별, 폭력 조장 등\n2. **직접적인 폭력·잔혹 행위**  \n   - 극단적인 폭력 묘사(살해·자해·테러 등)\n   - 노골적으로 잔혹하거나 충격적인 내용\n3. **노골적 성적 콘텐츠**  \n   - 명백히 선정적·음란한 묘사, 아동 성착취 등 불법적이거나 심각한 윤리적 문제가 있는 경우\n4. **사생활 침해, 불법 행위 조장**  \n   - 명백한 사생활 침해, 범죄 또는 불법 행위를 조장하는 내용\n5. **기타 위험 요소**  \n   - 위 항목 외에도, 사회적으로 유해하거나 심각한 해악을 유발할 수 있는 이미지·텍스트\n만약 콘텐츠가 위 기준 중 하나라도 충족하여 "유해"하다고 판단되면 "1"을,  \n해당되지 않으면 "0"을 출력하십시오.\n출력은 오직 숫자 하나(1 또는 0)만 반환하고, 어떠한 추가 문구나 설명도 첨부하지 마십시오.\n\n###텍스트 :시발 그녀'}, {'type': 'image', 'image': '/

In [26]:
# collate_fn 테스트 (배치 크기 1로)
batch = collate_fn([example])
print("\n처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("이미지 픽셀 형태:", batch["pixel_values"].shape)
print("레이블 형태:", batch["labels"].shape)


처리된 배치 데이터:
입력 ID 형태: torch.Size([1, 1109])
어텐션 마스크 형태: torch.Size([1, 1109])
이미지 픽셀 형태: torch.Size([2304, 1176])
레이블 형태: torch.Size([1, 1109])


In [30]:
print('입력에 대한 정수 인코딩 결과:')
print(batch["input_ids"][0])

입력에 대한 정수 인코딩 결과:
tensor([151644,   8948,    198,  ...,     16, 151645,    198])


In [31]:
print('레이블에 대한 정수 인코딩 결과:')
print(batch["labels"][0])

레이블에 대한 정수 인코딩 결과:
tensor([151644,   8948,    198,  ...,     16, 151645,    198])


In [27]:
# 토큰 디코딩 예시 (입력 텍스트가 어떻게 변환되었는지 확인)
decoded_text = processor.tokenizer.decode(batch["input_ids"][0])
print("\n디코딩된 텍스트:")
print(decoded_text)


디코딩된 텍스트:
<|im_start|>system
당신은 이미지와 텍스트의 '유해성(harmfulness)'을 판별하는 분류 모델입니다.<|im_end|>
<|im_start|>user
입력으로 이미지 또는 ###텍스트가 주어지면, 해당 콘텐츠가 유해한지 여부를 "1" 또는 "0"으로 출력하세요.
- 출력은 반드시 숫자 하나만(문자열 없이) 출력합니다.
  - "1" : 유해(혐오·차별·폭력·선정성·기타 해악 가능성이 있다고 판단되는 경우)
  - "0" : 비유해(문제 없다고 판단되는 경우)
유해성(harmfulness)을 판단할 때는 아래 항목을 포함해 검토하세요:
1. **혐오 표현 (Hate Speech)**  
   - 인종, 성별, 종교, 성적 지향, 장애 등을 근거로 특정 집단/개인을 비하·모욕·차별하는 언어/이미지
   - 명백한 증오, 차별, 폭력 조장 등
2. **직접적인 폭력·잔혹 행위**  
   - 극단적인 폭력 묘사(살해·자해·테러 등)
   - 노골적으로 잔혹하거나 충격적인 내용
3. **노골적 성적 콘텐츠**  
   - 명백히 선정적·음란한 묘사, 아동 성착취 등 불법적이거나 심각한 윤리적 문제가 있는 경우
4. **사생활 침해, 불법 행위 조장**  
   - 명백한 사생활 침해, 범죄 또는 불법 행위를 조장하는 내용
5. **기타 위험 요소**  
   - 위 항목 외에도, 사회적으로 유해하거나 심각한 해악을 유발할 수 있는 이미지·텍스트
만약 콘텐츠가 위 기준 중 하나라도 충족하여 "유해"하다고 판단되면 "1"을,  
해당되지 않으면 "0"을 출력하십시오.
출력은 오직 숫자 하나(1 또는 0)만 반환하고, 어떠한 추가 문구나 설명도 첨부하지 마십시오.

###텍스트 :시발 그녀<|vision_start|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|imag

In [28]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    data_collator=collate_fn,
    dataset_text_field="",
    peft_config=peft_config,
    tokenizer=processor.tokenizer,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overf

In [29]:
# 학습 시작, 모델은 자동으로 허브와 출력 디렉토리에 저장됨
trainer.train()

# 모델 저장
trainer.save_model(args.output_dir)

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]


Step,Training Loss
5,1.967100
10,1.764500
15,1.472200
20,1.164700
25,0.864800
30,0.578600
35,0.382700
40,0.256100
45,0.216400
50,0.195800


Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]
Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]
Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
Unrecognized keys in `rope_scaling` for 'r

In [ ]:
# 메모리 비우기
del model
del trainer
torch.cuda.empty_cache()

In [1]:
import torch
from transformers import AutoProcessor, AutoModelForVision2Seq

model_id = "Qwen/Qwen2-VL-7B-Instruct" 

# 기본 모델 호출
model = AutoModelForVision2Seq.from_pretrained(
  model_id,
  device_map="auto",
  torch_dtype=torch.float16
)
processor = AutoProcessor.from_pretrained(model_id)

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [2]:
# 로라 어댑터가 있는 경로
adapter_path1 = "/workspace/CookBook/chapter09-Multi_Modal/qwen2-7b-instruct-harmful-detector-2000/checkpoint-33"
adapter_path2 = "/workspace/CookBook/chapter09-Multi_Modal/qwen2-7b-instruct-harmful-detector-2000/checkpoint-67"
adapter_path3 = "/workspace/CookBook/chapter09-Multi_Modal/qwen2-7b-instruct-harmful-detector-2000/checkpoint-99"

# 첫 번째 Adapter 로드
model.load_adapter(
    adapter_path1,
    adapter_name="adapter1") 

In [15]:
from tqdm.auto import tqdm 
train_result = [] 

# adapter1로 테스트하기 
model.set_adapter("adapter1")
for idx in tqdm(range(len(test_dataset))):
    messages =  test_dataset[idx]["messages"]
    answer = test_dataset[idx]["messages"][2]["content"][0]["text"]
    ft_description = generate_description(messages, model, processor)
    train_result.append((int(answer), int(ft_description)))

after_prediction_result = [int(predict) == int(answer) for predict, answer in train_result]
after_train_accuracy = sum(after_prediction_result) /  len(after_prediction_result) * 100
print(f"학습 후 정확도 : {after_train_accuracy:.2f}%") 

  0%|          | 0/240 [00:00<?, ?it/s]

학습 후 정확도 : 96.67%


In [16]:
# 첫 번째 Adapter 로드
model.load_adapter(
    adapter_path2,
    adapter_name="adapter2") 

In [17]:
from tqdm.auto import tqdm 
train_result = [] 

# adapter2로 테스트하기 
model.set_adapter("adapter2")
for idx in tqdm(range(len(test_dataset))):
    messages =  test_dataset[idx]["messages"]
    answer = test_dataset[idx]["messages"][2]["content"][0]["text"]
    ft_description = generate_description(messages, model, processor)
    train_result.append((int(answer), int(ft_description)))

after_prediction_result = [int(predict) == int(answer) for predict, answer in train_result]
after_train_accuracy = sum(after_prediction_result) /  len(after_prediction_result) * 100
print(f"학습 후 정확도 : {after_train_accuracy:.2f}%") 

  0%|          | 0/240 [00:00<?, ?it/s]

학습 후 정확도 : 100.00%
